In [1]:
!pip install onnxruntime -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00


In [2]:
!pip install omegaconf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!apt update
!apt install -y aspell aspell-ru

In [4]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import subprocess
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from typing import List

import numpy as np
import onnxruntime as ort
from huggingface_hub import hf_hub_download
from omegaconf import OmegaConf
from sentencepiece import SentencePieceProcessor

In [6]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [7]:
sentences = pd.read_csv('sentences.csv')

In [8]:
sentences.head()

,id,document_id,sentence_index,text,corrected,status
0,1001,1,1,Загрязнение тяжелыми металлами Дальнегорского ...,Загрязнение тяжелыми металлами Дальнегорского ...,NaN
1,1002,1,2,Одной из самых главных экологических проблем н...,Одной из самых главных экологических проблем н...,NaN
2,1003,1,3,Эта проблема особеннo характерна для тех местн...,Эта проблема особеннo характерна для тех местн...,NaN
3,1004,1,4,Согласно проведенным исследованиям Тихоокеанск...,Согласно проведенным исследованиям Тихоокеанск...,NaN
4,1005,1,5,В ходе исследования было выявлено высокое соде...,В ходе исследования было выявлено высокое соде...,NaN


In [9]:
sentences = sentences[~sentences.corrected.isna()]

In [10]:
def spellchecker(text):
    command = "echo '{}' | aspell --lang=ru list".format(text.replace("'", "’"))
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    misspelled_words = process.stdout.read().strip().split('\n')

    corrected_text = text
    for word in misspelled_words:
        if word:
            suggest_command = f"echo '{word}' | aspell --lang=ru pipe"
            suggest_process = subprocess.Popen(suggest_command, shell=True, stdout=subprocess.PIPE, text=True)
            output = suggest_process.stdout.readlines()
            if len(output) > 1 and ":" in output[1]:
                suggestions = output[1].strip().split(": ")[1].split(", ")
                if suggestions and suggestions[0]:
                    corrected_text = corrected_text.replace(word, suggestions[0], 1)

    return corrected_text

In [11]:
sentences['text_orpho'] = sentences['text'].progress_apply(spellchecker)

  0%|          | 0/31490 [00:00<?, ?it/s]

In [12]:
sentences.head()

,id,document_id,sentence_index,text,corrected,status,text_orpho
0,1001,1,1,Загрязнение тяжелыми металлами Дальнегорского ...,Загрязнение тяжелыми металлами Дальнегорского ...,NaN,Загрязнение тяжелыми металлами Дальнозоркого р...
1,1002,1,2,Одной из самых главных экологических проблем н...,Одной из самых главных экологических проблем н...,NaN,Одной из самых главных экологических проблем н...
2,1003,1,3,Эта проблема особеннo характерна для тех местн...,Эта проблема особеннo характерна для тех местн...,NaN,Эта проблема особенно характерна для тех местн...
3,1004,1,4,Согласно проведенным исследованиям Тихоокеанск...,Согласно проведенным исследованиям Тихоокеанск...,NaN,Согласно проведенным исследованиям Тихоокеанск...
4,1005,1,5,В ходе исследования было выявлено высокое соде...,В ходе исследования было выявлено высокое соде...,NaN,В ходе исследования было выявлено высокое соде...


In [13]:
def cleaning(punct, texts):
  clean_data = []
  for text in texts:
    for item in punct:
      if item in text:
        text = text.replace(item, '')
    clean_text = ' '.join([word for word in word_tokenize(text)])
    clean_data.append(clean_text)
  return clean_data


clean_df = pd.DataFrame({'cleaned_text': cleaning(string.punctuation, sentences.text_orpho), 'cleaned_target': cleaning(string.punctuation, sentences.corrected)})
clean_df

,cleaned_text,cleaned_target
0,Загрязнение тяжелыми металлами Дальнозоркого р...,Загрязнение тяжелыми металлами Дальнегорского ...
1,Одной из самых главных экологических проблем н...,Одной из самых главных экологических проблем н...
2,Эта проблема особенно характерна для тех местн...,Эта проблема особеннo характерна для тех местн...
3,Согласно проведенным исследованиям Тихоокеанск...,Согласно проведенным исследованиям Тихоокеанск...
4,В ходе исследования было выявлено высокое соде...,В ходе исследования было выявлено высокое соде...
...,...,...
31485,Он действительно хорошо работает с людьми пото...,Он действительно хорошо работает с людьми пото...
31486,Я бы сказала много хорошего о Джоне за весь год,Я бы сказала много хорошего о Джоне за весь год
31487,Он честно говоря хороший человек,Он честно говоря хороший человек
31488,Надеюсь я ответила на все твоя вопросы не забу...,Надеюсь я ответила на все твои вопросы не забу...


In [14]:
# the code is taken from here: https://huggingface.co/1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase
# Download the models from HF hub. Note: to clean up, you can find these files in your HF cache directory
spe_path = hf_hub_download(repo_id="1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase", filename="sp.model")
onnx_path = hf_hub_download(repo_id="1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase", filename="model.onnx")
config_path = hf_hub_download(
    repo_id="1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase", filename="config.yaml"
)

# Load the SP model
tokenizer: SentencePieceProcessor = SentencePieceProcessor(spe_path)  # noqa
# Load the ONNX graph
ort_session: ort.InferenceSession = ort.InferenceSession(onnx_path)
# Load the model config with labels, etc.
config = OmegaConf.load(config_path)
# Potential classification labels before each subtoken
pre_labels: List[str] = config.pre_labels
# Potential classification labels after each subtoken
post_labels: List[str] = config.post_labels
# Special class that means "predict nothing"
null_token = config.get("null_token", "<NULL>")
# Special class that means "all chars in this subtoken end with a period", e.g., "am" -> "a.m."
acronym_token = config.get("acronym_token", "<ACRONYM>")
# Not used in this example, but if your sequence exceed this value, you need to fold it over multiple inputs
max_len = config.max_length
# For reference only, graph has no language-specific behavior
languages: List[str] = config.languages

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sp.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/531 [00:00<?, ?B/s]

In [15]:
def add_punct(input_text,
              tokenizer=tokenizer,
              ort_session=ort_session,
              config=config,
              pre_labels=pre_labels,
              post_labels=post_labels,
              null_token=null_token,
              acronym_token=acronym_token,
              max_len=max_len,
              languages=languages) -> str:

    # Encode some input text, adding BOS + EOS
    input_ids = [tokenizer.bos_id()] + tokenizer.EncodeAsIds(input_text) + [tokenizer.eos_id()]

    # Create a numpy array with shape [B, T], as the graph expects as input.
    # Note that we do not pass lengths to the graph; if you are using a batch, padding should be tokenizer.pad_id() and the
    # graph's attention mechanisms will ignore pad_id() without requiring explicit sequence lengths.
    input_ids_arr: np.array = np.array([input_ids])

    # Run the graph, get outputs for all analytics
    pre_preds, post_preds, cap_preds, sbd_preds = ort_session.run(None, {"input_ids": input_ids_arr})
    # Squeeze off the batch dimensions and convert to lists
    pre_preds = pre_preds[0].tolist()
    post_preds = post_preds[0].tolist()
    cap_preds = cap_preds[0].tolist()
    sbd_preds = sbd_preds[0].tolist()

    # Segmented sentences
    output_texts: List[str] = []
    # Current sentence, which is built until we hit a sentence boundary prediction
    current_chars: List[str] = []
    # Iterate over the outputs, ignoring the first (BOS) and final (EOS) predictions and tokens
    for token_idx in range(1, len(input_ids) - 1):
        token = tokenizer.IdToPiece(input_ids[token_idx])
        # Simple SP decoding
        if token.startswith("▁") and current_chars:
            current_chars.append(" ")
        # Token-level predictions
        pre_label = pre_labels[pre_preds[token_idx]]
        post_label = post_labels[post_preds[token_idx]]
        # If we predict "pre-punct", insert it before this token
        if pre_label != null_token:
            current_chars.append(pre_label)
        # Iterate over each char. Skip SP's space token,
        char_start = 1 if token.startswith("▁") else 0
        for token_char_idx, char in enumerate(token[char_start:], start=char_start):
            # If this char should be capitalized, apply upper case
            if cap_preds[token_idx][token_char_idx]:
                char = char.upper()
            # Append char
            current_chars.append(char)
            # if this is an acronym, add a period after every char (p.m., a.m., etc.)
            if post_label == acronym_token:
                current_chars.append(".")
        # Maybe this subtoken ends with punctuation
        if post_label != null_token and post_label != acronym_token:
            current_chars.append(post_label)

        # If this token is a sentence boundary, finalize the current sentence and reset
        if sbd_preds[token_idx]:
            output_texts.append("".join(current_chars))
            current_chars.clear()

    # Maybe push final sentence, if the final token was not classified as a sentence boundary
    if current_chars:
        output_texts.append("".join(current_chars))
    return ' '.join(output_texts)

In [16]:
def tokenization_with_punct(text):
    text_with_punct = add_punct(text)
    tok_text = ' '.join([word for word in word_tokenize(text_with_punct)])
    return tok_text


clean_df['text'] = clean_df['cleaned_text'].progress_apply(lambda x: tokenization_with_punct(x))
clean_df['corrected'] = clean_df['cleaned_target'].progress_apply(lambda x: tokenization_with_punct(x))
clean_df.head()

  0%|          | 0/31490 [00:00<?, ?it/s]

  0%|          | 0/31490 [00:00<?, ?it/s]

,cleaned_text,cleaned_target,text,corrected
0,Загрязнение тяжелыми металлами Дальнозоркого р...,Загрязнение тяжелыми металлами Дальнегорского ...,Загрязнение тяжелыми металлами Дальнозоркого р...,Загрязнение тяжелыми металлами Дальнегорского ...
1,Одной из самых главных экологических проблем н...,Одной из самых главных экологических проблем н...,Одной из самых главных экологических проблем н...,Одной из самых главных экологических проблем н...
2,Эта проблема особенно характерна для тех местн...,Эта проблема особеннo характерна для тех местн...,Эта проблема особенно характерна для тех местн...,Эта проблема особеннo характерна для тех местн...
3,Согласно проведенным исследованиям Тихоокеанск...,Согласно проведенным исследованиям Тихоокеанск...,Согласно проведенным исследованиям Тихоокеанск...,Согласно проведенным исследованиям Тихоокеанск...
4,В ходе исследования было выявлено высокое соде...,В ходе исследования было выявлено высокое соде...,В ходе исследования было выявлено высокое соде...,В ходе исследования было выявлено высокое соде...


In [17]:
clean_df[['text', 'corrected']].to_csv('sentences_preprocessed.csv', index=False)